<a href="https://colab.research.google.com/github/nikhilgunti/Skin-Cancer-Detection/blob/master/Deep_learning_CNN_Skin_cancer_images_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from swat import CAS
import swat
from swat.cas.table import merge
import os
import dlpy
import numpy
import matplotlib.image as mping
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import pandas as pd

from dlpy.images import ImageTable
from dlpy.applications import UNet
from dlpy.model import Optimizer, Model, DataSpec, MomentumSolver, AdamSolver, StepLR
from dlpy.layers import *
from dlpy.utils import display_segmentation_images, create_segmentation_table, display_segmentation_results
from dlpy.sequential import Sequential

In [0]:
cashost='rdcgrd001'
casport=29564
casauth='C:/Users/nijakh/_authinfo'

# Start CAS session
conn = CAS(cashost, casport, authinfo=casauth, caslib="casuser")
conn.loadactionset('image')
conn.loadactionset('deeplearn')

NOTE: Added action set 'image'.
NOTE: Added action set 'deeplearn'.


[actionset]

 'deeplearn'

+ Elapsed: 0.0785s, user: 0.302s, sys: 0.639s, mem: 30.9mb

In [0]:
# load_data
image_data=conn.loadTable( caslib="casuser",path="hmnist_28_28_L.csv", importoptions={"filetype":"csv"},casout={"name":"hmnist"})


NOTE: Cloud Analytic Services made the file hmnist_28_28_L.csv available as table HMNIST in caslib CASUSERHDFS(nijakh).


In [0]:
#Split data into train and test
conn.loadactionset('sampling')
conn.sampling.stratified(table="hmnist",target='label',samppct=20,samppct2=10,output={"casout":{"name":"hmnist_sample","replace":True},"copyvars":"ALL"})

NOTE: Added action set 'sampling'.
NOTE: The PARTIND option is set to be TRUE for segment stratified sampling.
NOTE: Stratified sampling is in effect.
NOTE: Using SEED=595555116 for sampling.


[OutputCasTables]

                 casLib           Name Label   Rows  Columns  \
 0  CASUSERHDFS(nijakh)  hmnist_sample        10015      786   
 
                                             casTable  
 0  CASTable('hmnist_sample', caslib='CASUSERHDFS(...  

[PartIndMap]

 Partition Indicator Map
 
   PartIndName TargetName
 0  _PartInd1_      label

[STRAFreq]

 Frequencies
 
   TargetName TargetLevel   NObs  NSamp1  NSamp2
 0      label              10015    2003    1002

+ Elapsed: 4.04s, user: 15s, sys: 18.6s, mem: 1.15e+04mb

In [0]:
conn.loadactionset('table')
conn.table.save(caslib="casuser",name="hmnist_train",replace=True,table={"name":'hmnist_sample',"where":"_partind1_=0" })
conn.table.save(caslib="casuser",name="hmnist_valid",replace=True,table={"name":'hmnist_sample',"where":"_partind1_=1" })
conn.table.save(caslib="casuser",name="hmnist_test",replace=True,table={"name":'hmnist_sample',"where":"_partind1_=2" })

NOTE: Added action set 'table'.
NOTE: Cloud Analytic Services saved the file hmnist_train.sashdat in caslib CASUSER(nijakh).
NOTE: Cloud Analytic Services saved the file hmnist_valid.sashdat in caslib CASUSER(nijakh).
NOTE: Cloud Analytic Services saved the file hmnist_test.sashdat in caslib CASUSER(nijakh).


[caslib]

 'CASUSER(nijakh)'

[name]

 'hmnist_test.sashdat'

+ Elapsed: 0.306s, user: 1.59s, sys: 1.13s, mem: 2.78e+03mb

In [0]:
# load_data
image_data=conn.loadTable( caslib="casuser",path="hmnist_train.sashdat", importoptions={"filetype":"hdat"},casout={"name":"train","replace":True})
image_data=conn.loadTable( caslib="casuser",path="hmnist_valid.sashdat", importoptions={"filetype":"hdat"},casout={"name":"valid","replace":True})
image_data=conn.loadTable( caslib="casuser",path="hmnist_test.sashdat", importoptions={"filetype":"hdat"},casout={"name":"test","replace":True})


NOTE: Cloud Analytic Services made the file hmnist_train.sashdat available as table TRAIN in caslib CASUSERHDFS(nijakh).
NOTE: Cloud Analytic Services made the file hmnist_valid.sashdat available as table VALID in caslib CASUSERHDFS(nijakh).
NOTE: Cloud Analytic Services made the file hmnist_test.sashdat available as table TEST in caslib CASUSERHDFS(nijakh).


In [0]:
conn.table.tableInfo(name="test")      
#conn.table.tableDetails(name="test")

,Name,Rows,Columns,IndexedColumns,Encoding,CreateTimeFormatted,ModTimeFormatted,AccessTimeFormatted,JavaCharSet,CreateTime,...,Repeated,View,MultiPart,SourceName,SourceCaslib,Compressed,Creator,Modifier,SourceModTimeFormatted,SourceModTime
0,TEST,1002,786,0,utf-8,2020-03-23T11:15:48-04:00,2020-03-23T11:15:48-04:00,2020-03-23T11:15:48-04:00,UTF8,1.900596e+09,...,0,0,0,hmnist_test.sashdat,CASUSER(nijakh),0,nijakh,,2020-03-23T11:15:46-04:00,1.900596e+09


In [0]:
#define empty cnn model
conn.deepLearn.buildModel(
modelTable={"name":"hmnistcnn", "replace":True},
type="CNN")

conn.deepLearn.modelInfo(                        
modelTable={"name":"hmnistcnn"}
)
  

,Descr,Value
0,Model Name,hmnistcnn
1,Model Type,Convolutional Neural Network
2,Number of Layers,0
3,Number of Input Layers,0
4,Number of Output Layers,0
5,Number of Convolutional Layers,0
6,Number of Pooling Layers,0
7,Number of Fully Connected Layers,0


In [0]:
#Add layers to empty cnn model

#Add input layer
conn.deepLearn.addLayer(          
layer={"type":"INPUT","height":28,
"width":28},    
modelTable={"name":"hmnistcnn"} ,   
name="data")           

#Conv layer with 32 filters of 3 by 3
conn.deepLearn.addLayer(
layer={"type":"convo",
#"act":"identity",
"nfilters":32,
"height":5,
"width":5,
"dropout":.3    
},      
modelTable={"name":"hmnistcnn"} ,  
name="conv1",
srcLayers={"data"})   

#batch normalization layer 
conn.deepLearn.addLayer(
layer={"type":"batchnorm"} ,     
modelTable={"name":"hmnistcnn"} ,  
name="batch1",
srcLayers={"conv1"})  

#pooling layer 
conn.deepLearn.addLayer(
layer={"type":"pool","dropout":.3,"height":2, "width":2}  ,    
modelTable={"name":"hmnistcnn"}   ,
name="pool1",
srcLayers={"batch1"})

#Conv layer with 64 filters of 3 by 3
conn.deepLearn.addLayer(
layer={"type":"convo",
#"act":"identity",
"nfilters":64,
"height":3,
"width":3,
"dropout":.3,    
}      ,
modelTable={"name":"hmnistcnn"},   
name="conv2",
srcLayers={"pool1"})   

#batch normalization layer 
conn.deepLearn.addLayer(
layer={"type":"batchnorm"}  ,    
modelTable={"name":"hmnistcnn"} ,  
name="batch2",
srcLayers={"conv2"})  

#pooling layer 
conn.deepLearn.addLayer(
layer={"type":"pool","dropout":.3,"height":2, "width":2}   ,   
modelTable={"name":"hmnistcnn"}  , 
name="pool2",
srcLayers={"batch2"})

#output layer
conn.deepLearn.addLayer(
layer={"type":"output",
"act":'softmax',
"fullConnect":True
},    
modelTable={"name":"hmnistcnn"}  ,   
name="outlayer",
srcLayers={"pool2"})           


,casLib,Name,Rows,Columns,casTable
0,CASUSERHDFS(nijakh),hmnistcnn,98,5,"CASTable('hmnistcnn', caslib='CASUSERHDFS(nija..."


In [0]:
conn.deepLearn.modelInfo(                        
modelTable={"name":"hmnistcnn"}
)

,Descr,Value
0,Model Name,hmnistcnn
1,Model Type,Convolutional Neural Network
2,Number of Layers,8
3,Number of Input Layers,1
4,Number of Output Layers,1
5,Number of Convolutional Layers,2
6,Number of Pooling Layers,2
7,Number of Fully Connected Layers,0
8,Number of Batch Normalization Layers,2


In [0]:
#create input variable list 
vars=[]
for x in range(784):
    tmpx=str(x).zfill(4)
    vname="pixel{0}".format(tmpx)
    vars.append(vname)
    
#vars     

In [0]:
#Train the model
conn.deepLearn.dlTrain(
optimizer={"algorithm":{"method":"ADAM"}, "minibatchsize":64, "maxepochs":50},
inputs=vars,
modelTable={"name":"hmnistcnn"},
modelWeights={"name":"hmnistTrainedWeights",
"replace":True},
table={"name":"train"},
nominals=[{"name":"label"}],
target="label",
validtable={"name":"valid"})


,Descr,Value
0,Model Name,hmnistcnn
1,Model Type,Convolutional Neural Network
2,Number of Layers,8
3,Number of Input Layers,1
4,Number of Output Layers,1
5,Number of Convolutional Layers,2
6,Number of Pooling Layers,2
7,Number of Fully Connected Layers,0
8,Number of Batch Normalization Layers,2
9,Number of Weight Parameters,41184


In [0]:
conn.deepLearn.dlScore(                
      casOut={"name":"test_scored",
              "replace":True},copyvars={"label"},
      initWeights={"name":"hmnistTrainedWeights"},
      modelTable={"name":"hmnistcnn"},
      table={"name":"test"}
)

[OutputCasTables]

                 casLib         Name  Rows  Columns  \
 0  CASUSERHDFS(nijakh)  test_scored  1002       11   
 
                                             casTable  
 0  CASTable('test_scored', caslib='CASUSERHDFS(ni...  

[ScoreInfo]

                          Descr         Value
 0  Number of Observations Read          1002
 1  Number of Observations Used          1002
 2  Misclassification Error (%)      33.63273
 3                   Loss Error      1.297432

+ Elapsed: 2.32s, user: 15.7s, sys: 48.8s, mem: 2.59e+04mb

In [0]:
#test_out=conn.CASTable('test_scored')
#test_out.fetch(maxrows=100)
conn.table.fetch(table="test_scored",maxrows=100,to=1000)

,label,_DL_P0_,_DL_P1_,_DL_P2_,_DL_P3_,_DL_P4_,_DL_P5_,_DL_P6_,_DL_PredName_,_DL_PredP_,_DL_PredLevel_
0,2.0,0.076492,0.228905,0.142519,0.065847,0.245478,0.066110,0.174649,0,0.245478,4.0
1,2.0,0.043153,0.290850,0.215652,0.089837,0.104826,0.043637,0.212045,4,0.290850,1.0
2,2.0,0.082637,0.222707,0.105718,0.071870,0.250666,0.094852,0.171550,0,0.250666,4.0
3,2.0,0.063762,0.274222,0.111230,0.074859,0.207827,0.094565,0.173535,4,0.274222,1.0
4,2.0,0.073605,0.320467,0.100352,0.057217,0.193148,0.091947,0.163264,4,0.320467,1.0
...,...,...,...,...,...,...,...,...,...,...,...
95,1.0,0.053347,0.285169,0.097200,0.059793,0.248420,0.085591,0.170480,4,0.285169,1.0
96,1.0,0.039238,0.170415,0.133730,0.085597,0.277204,0.078176,0.215640,0,0.277204,4.0
97,1.0,0.035224,0.192747,0.341358,0.036175,0.111312,0.018612,0.264572,6,0.341358,2.0
98,1.0,0.023712,0.195719,0.164585,0.067829,0.288895,0.076008,0.183252,0,0.288895,4.0


In [0]:
conn.loadactionset('freqtab')
conn.freqtab.freqtab(table="test_scored",tabulate=[{"vars":{"label"},"cross":{"_DL_PredLevel_"}}])

NOTE: Added action set 'freqtab'.


[LevelInfo]

 Variable Level Information
 
          Variable                                     Label  Levels  \
 0  _DL_PredLevel_  Predicted level with highest probability       5   
 1           label                                                 7   
 
                                               Values  
 0             0            1            2        ...  
 1                                      0 1 2 3 4 5 6  

[NObs]

 Number of Observations
 
       RowId                  Description   Value
 0     NREAD  Number of Observations Read  1002.0
 1     NUSED  Number of Observations Used  1002.0
 2  NWEIGHTS               Sum of Weights  1002.0

[Table1.CrossList]

 Table of label by _DL_PredLevel_
 
    F_label  label F__DL_PredLevel_  _DL_PredLevel_  Frequency     Percent  \
 0        0    0.0                1             1.0       20.0    1.996008   
 1        0    0.0                2             2.0        2.0    0.199601   
 2        0    0.0                4             4.0        8.0    0.798403   
 3        0    0.0                6             6.0        1.0    0.099800   
 4        0    0.0            Total             NaN       31.0    3.093812   
 5        1    1.0                1             1.0       41.0    4.091816   
 6        1    1.0                2             2.0        3.0    0.299401   
 7        1    1.0                4             4.0       11.0    1.097804   
 8        1    1.0            Total             NaN       55.0    5.489022   
 9        2    2.0                1             1.0       76.0    7.584830   
 10       2    2.0                2             2.0        8.0    0.798403   
 11       2    2.0                4             4.0       23.0    2.295409   
 12       2    2.0                6             6.0        4.0    0.399202   
 13       2    2.0            Total             NaN      111.0   11.077844   
 14       3    3.0                1             1.0        9.0    0.898204   
 15       3    3.0                4             4.0        2.0    0.199601   
 16       3    3.0                6             6.0        2.0    0.199601   
 17       3    3.0            Total             NaN       13.0    1.297405   
 18       4    4.0                1             1.0      639.0   63.772455   
 19       4    4.0                2             2.0       15.0    1.497006   
 20       4    4.0                4             4.0       16.0    1.596806   
 21       4    4.0                6             6.0        1.0    0.099800   
 22       4    4.0            Total             NaN      671.0   66.966068   
 23       5    5.0                1             1.0       11.0    1.097804   
 24       5    5.0                2             2.0        1.0    0.099800   
 25       5    5.0                4             4.0        1.0    0.099800   
 26       5    5.0            Total             NaN       13.0    1.297405   
 27       6    6.0                0             0.0        2.0    0.199601   
 28       6    6.0                1             1.0       83.0    8.283433   
 29       6    6.0                2             2.0       14.0    1.397206   
 30       6    6.0                4             4.0        6.0    0.598802   
 31       6    6.0                6             6.0        3.0    0.299401   
 32       6    6.0            Total             NaN      108.0   10.778443   
 33   Total    NaN                0             0.0        2.0    0.199601   
 34   Total    NaN                1             1.0      879.0   87.724551   
 35   Total    NaN                2             2.0       43.0    4.291417   
 36   Total    NaN                4             4.0       67.0    6.686627   
 37   Total    NaN                6             6.0       11.0    1.097804   
 38   Total    NaN            Total             NaN     1002.0  100.000000   
 
     RowPercent  ColPercent  
 0    64.516129    2.275313  
 1     6.451613    4.651163  
 2    25.806452   11.940299  
 3     3.225806    9.090909  
 4   10